In [15]:
# import libraries
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd 
import plotly.express as px

In [16]:
df = pd.read_csv('Data/2018-2020_complete_df.csv')

In [17]:
df.head(5)

,year,locationabbr,locationdesc,data_value,Gender,race/ethnicityage_group,locationid,y_lat,x_lon
0,2019,AK,Aleutians East,182.4,Overall,Overall,2013,55.442393,-161.959930
1,2019,AK,Aleutians West,172.6,Overall,Overall,2016,53.648343,-166.917533
2,2019,AK,Anchorage,255.6,Overall,Overall,2020,61.159145,-149.104559
3,2019,AK,Bethel,343.4,Overall,Overall,2050,60.923648,-159.753262
4,2019,AK,Bristol Bay,NaN,Overall,Overall,2060,58.753496,-156.694662


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59076 entries, 0 to 59075
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     59076 non-null  int64  
 1   locationabbr             59076 non-null  object 
 2   locationdesc             59076 non-null  object 
 3   data_value               33069 non-null  float64
 4   Gender                   59076 non-null  object 
 5   race/ethnicityage_group  59076 non-null  object 
 6   locationid               59076 non-null  int64  
 7   y_lat                    59076 non-null  float64
 8   x_lon                    59076 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 4.1+ MB


## QUESTION 1: Số người bị đột quỵ phân bố như thế nào trong các bang đối với nhóm người phụ nữ - da trắng?

In [19]:
df_Female_White= df[(df['Gender'] == 'Female') & (df['race/ethnicityage_group'] == 'White')]

In [20]:
df_Female_White = df_Female_White[['locationabbr', 'locationdesc','y_lat', 'x_lon', 'data_value']]
df_Female_White_gr = df_Female_White.groupby('locationabbr').agg({'y_lat': 'first', 'x_lon': 'first', 'data_value': 'mean'}).reset_index()
df_Female_White_gr.head(5)

,locationabbr,y_lat,x_lon,data_value
0,AK,55.442393,-161.959930,181.333333
1,AL,32.539426,-86.641296,354.176471
2,AR,34.291259,-91.367868,340.700000
3,AS,-14.301754,-170.719474,NaN
4,AZ,35.394866,-109.486870,228.775000


In [22]:
# Tạo custom color scale từ màu đỏ nhạt đến màu đỏ đậm
custom_scale = [
    [0, 'rgb(255, 255, 255)'],  # Màu trắng cho giá trị nhỏ
    [150 / df_Female_White_gr['data_value'].max(), 'rgb(255, 230, 230)'],  # Màu đỏ nhạt cho giá trị < 200
    [1, 'rgb(255,0,0)']  # Màu đỏ đậm cho giá trị > 150
]

fig = px.choropleth(   
    df_Female_White_gr,
    color=df_Female_White_gr['data_value'],
    locations=df_Female_White_gr['locationabbr'],
    locationmode="USA-states",
    scope="usa",
    color_continuous_scale=custom_scale,
    range_color=[df_Female_White_gr['data_value'].min(), df_Female_White_gr['data_value'].max()],  # Đảm bảo mọi giá trị đều được hiển thị
    hover_name=df_Female_White_gr['locationabbr'],
    title='Phân bố ca bệnh theo các bang của nước Mỹ',
    width=1000,
    height=600,
)

fig.show()


**Nhận xét:**
- Tại bang OK (Oklahoma) có giá trị data_value lớn nhất (283)  

**Lợi ích:**
- Phân tích tình hình xu hướng những năm gần đây có những ảnh hưởng gì dẫn tới sự tăng mạnh của data_value.
- Tìm hiểu những lý do giúp các năm 2009, 2010 có data_value đạt đáy để áp dụng.